# SHIFT_ML_2026_COMPETITION

## Описание задачи

Данные представляют собой более миллиона записей с более чем 100 признаками и одной целевой переменной: «итоговый_статус_займа». Значения: 0 – выплачен, 1 – не выплачен. Задача – бинарная классификация.

В вашем распоряжении будет два датасета: один тренировочный (с целевой переменной), и один тестовый – без целевой переменной.

Также в архиве находится ноутбук baseline.ipynb с примером подготовки файла с ответом.

Постановка задачи

Вам предстоит помочь банку принять решение о выдаче займа клиентам. Помните, что одного лишь принятого решения мало – нужно уметь интерпретировать полученные результаты.

Задание будет состоять из двух частей

1 часть – тестовая

Вам необходимо пройти тест, ссылка на который есть в письме с заданием. Можете сделать это до или после соревнования: порядок выполнения вы выбираете на ваше усмотрение.

2 часть – соревнование

Ваша задача – обучить классификатор, наилучшим образом предсказывающий целевую переменную. Помимо этого, обратите внимание на следующие важные моменты:

* EDA (исследование данных, гипотезы, визуализации)
* Качество кода (скорость алгоритмов, читаемость, знание и использование библиотечных функций)
* Комментарии (пояснения к действиям, объяснение принятых решений)
* Подбор модели (аргументированный выбор той или иной модели, валидация)
* Скор (то, насколько хороший получится результат)
* requirements.txt (посмотрите заранее, как должен выглядеть этот файл и из чего состоять)
* Формирование и сохранение submisson.csv должно происходить в ноутбуке
* Ноутбук должен быть полностью воспроизводимым и выдавать submisson.csv аналогичный тому, что вы загрузили на платформу
* Не забудьте отобразить ваш сабмит на лидерборде!
* Загружайте на платформу архив в формате submisson.zip, в котором должен находиться submisson.csv файл следующего формата:

ID	Proba
1	0.78
2	0.23
3	0.91
4	0.45
5	0.67
Где id – id клиента из тестового датасета, а proba – проба вашей модели.

Истинные значения целевой переменной известны только организаторам конкурса. Решения проверяются автоматически путем сопоставления с истинными значениями.

Итоговый результат
От вас нам нужен сабмит на платформу – архив submisson.zip, в котором находятся следующие файлы:

1. Прогноз модели в формате submisson.csv

2. Ноутбук с решением конкурса, EDA и формированием файла с ответами – competition.ipynb

3. Файл requirements.txt

Не забудьте добавить ваш сабмит на лидерборд! (Зеленая кнопка справа на сабмите)

Вопросы и обсуждения
Используйте вкладку форум чтобы задавать вопросы по тесту и соревнованию.

Дедлайн
Соревнование и прием ответов на тест заканчиваются 09.02.2026 в 00.00 по Новосибирскому времени. (UTC+7)

Будьте внимательны! Вам доступно 3 сабмита в сутки. Это значит, что вы не сможете отправить своё решение в четвёртый раз за день, поэтому, используйте доступные вам попытки с умом.

Указывайте при загрузке решения ваши реальные фамилии и имена на латинице, никнеймы учитываться не будут, например, так IvanovIvan PetrovaOlga
Подтверждение участия в соревновании происходит на вкладке My_submissions
Метрика
Метрика соревнования – roc-auc.

## Импорт библиотек

In [13]:
import re
import shap
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import Ridge
from catboost import CatBoostClassifier
from optuna import distributions
from optuna.integration import OptunaSearchCV
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

## Загрузка данных

In [14]:
test_path = './shift_ml_2026_test.csv'
train_path = './shift_ml_2026_train.csv'
test_df = pd.read_csv(test_path)
train_df = pd.read_csv(train_path)
print('train')
print(train_df.shape)
display(train_df.head())
print('test')
print(test_df.shape)
display(test_df.head())
print('y_train')
print(train_df['итоговый_статус_займа'].value_counts())

C:\Temp\ipykernel_19292\2014380920.py:4: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(train_path)


train
(1210779, 109)


,id,сумма_займа,срок_займа,процентная_ставка,аннуитет,рейтинг,допрейтинг,профессия_заемщика,стаж,владение_жильем,...,процент_счетов_прев_75_лимита,кол-во_публ_банкротств,кол-во_залогов,кредитный_лимит,кредитный_баланс_без_ипотеки,лимит_по_картам,лимит_по_аннуитетным_счетам,кредитный_баланс_по_возоб_счетам,особая_ситуация,тип_предоставления_кредита
0,68355089,1235000.0,3 года,11.99,41014.0,В,В1,инженер,10+ лет,ИПОТЕКА,...,7.7,0.0,0.0,15700850.0,1973750.0,3965000.0,1233350.0,NaN,Нет,Наличные
1,68341763,1000000.0,5 лет,10.78,21633.0,Б,Б4,водитель грузовика,10+ лет,ИПОТЕКА,...,50.0,0.0,0.0,10920900.0,934800.0,310000.0,743850.0,NaN,Нет,Наличные
2,68426831,597500.0,3 года,13.44,20259.0,В,В3,ветеринарный техник,4 года,АРЕНДА,...,100.0,0.0,0.0,845000.0,639900.0,470000.0,200000.0,NaN,Нет,Наличные
3,68476668,1000000.0,3 года,9.17,31879.0,Б,Б2,вице-президент операций по набору персонала,10+ лет,ИПОТЕКА,...,100.0,0.0,0.0,19442600.0,5838100.0,1575000.0,2322600.0,NaN,Нет,Наличные
4,67275481,1000000.0,3 года,8.49,31563.0,Б,Б1,дорожному водителю,10+ лет,ИПОТЕКА,...,0.0,0.0,0.0,9669500.0,1396850.0,725000.0,1807200.0,NaN,Нет,Наличные


test
(134531, 108)


,id,сумма_займа,срок_займа,процентная_ставка,аннуитет,рейтинг,допрейтинг,профессия_заемщика,стаж,владение_жильем,...,процент_счетов_прев_75_лимита,кол-во_публ_банкротств,кол-во_залогов,кредитный_лимит,кредитный_баланс_без_ипотеки,лимит_по_картам,лимит_по_аннуитетным_счетам,кредитный_баланс_по_возоб_счетам,особая_ситуация,тип_предоставления_кредита
0,85540387,450000.0,3 года,9.49,14413.0,Б,Б2,обслуживание клиентов,10+ лет,ИПОТЕКА,...,75.0,0.0,0.0,4282850.0,1180600.0,725000.0,1022000.0,NaN,Нет,Наличные
1,28112500,400000.0,3 года,6.03,12174.5,А,А1,помощник по правовым вопросам,5 лет,АРЕНДА,...,57.1,0.0,0.0,3340900.0,2381050.0,1260000.0,1548550.0,NaN,Нет,Наличные
2,65731570,1250000.0,3 года,12.05,41548.0,В,В1,специалист по анализу кредитоспособности,5 лет,ИПОТЕКА,...,16.7,0.0,0.0,11350200.0,1980650.0,1690000.0,1804450.0,NaN,Нет,Наличные
3,65874747,977500.0,5 лет,20.99,26439.5,Д,Д5,специальный специалист,3 года,ИПОТЕКА,...,66.7,1.0,0.0,9976550.0,1678600.0,470000.0,2139950.0,NaN,Нет,Наличные
4,57893355,520000.0,3 года,18.25,18865.0,Д,Д1,руководитель районного проекта,3 года,АРЕНДА,...,66.7,0.0,0.0,1953950.0,1272750.0,1125000.0,603950.0,NaN,Нет,Наличные


y_train
итоговый_статус_займа
0    969085
1    241694
Name: count, dtype: int64


Имеется дисбаланс в данных

## Предобработка данных

Разбиение на `train` и `val`

In [15]:
target = 'итоговый_статус_займа'
y = train_df[target]
X = train_df.drop(columns=[target])

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)


### Доля пропусков в данных

Рассчитываем долю пропущенных значений как для тренировочной, так и для тестовой выборок. 

Сортируем по убыванию доли пропущенных значений

In [16]:
missing_train = X_train.isna().mean()
missing_test  = test_df.isna().mean()

missing_table = pd.DataFrame({
    "train": missing_train,
    "test": missing_test
})

missing_table["diff"] = (missing_table.train - missing_table.test).abs()
with pd.option_context("display.max_rows", None, 
                       "display.max_columns", None,
                       "display.float_format", '{:.4f}'.format):
    display(missing_table.sort_values("train", ascending=False).head(30))

,train,test,diff
дата_следующей_выплаты,1.0000,1.0000,0.0000
кредитный_баланс_по_возоб_счетам,0.9862,0.9865,0.0003
совокупный_статус_подтверждения_доходов_заемщиков,0.9811,0.9811,0.0000
совокупный_пдн_заемщиков,0.9809,0.9809,0.0000
совокупный_доход_заемщиков,0.9809,0.9809,0.0000
кол-во_месяцев_с_последнего_займа,0.8301,0.8298,0.0003
кол-во_мес_с_последней_задолженности_по_карте,0.7627,0.7639,0.0013
кол-во_месяцев_с_последнего_нарушения,0.7369,0.7392,0.0023
кол-во_мес_с_последней_задолженности_по_возобновляемому_счету,0.6653,0.6671,0.0018
соотношение_сумм_текущего_баланса_к_лимиту_по_аннуитетным_счетам,0.6545,0.6534,0.0011


Как мы можем видеть, доля пропущенных значений изменяется от 0 до 1, что закономерно. На данный момент сложно сказать, что делать с пропусками, анализ этого момента будет ниже. Есть столбцы, которые нужно явно удалить:
* `дата_следующей_выплаты` - нет значений
* `пени_за_дефолт` - нет значений в `test`

Удаление полей, которые не несут в себе полезной информации для модели, т. е.:
* Все значения пропущены в `train` или в `test`
* Сильно различается доля пропусков в `train` и `test`

Также в этом месте удалим из анализа поле `id`, поскольку оно не является характеристикой клиента или кредита

In [17]:
drop_cols = []
drop_cols.append('id')

for col in X_train.columns:
    if missing_train[col] == 1:
        drop_cols.append(col)
    elif col in missing_test and abs(missing_train[col]-missing_test[col]) > 0.3:
        drop_cols.append(col)
    elif X_train[col].nunique() <= 1:
        drop_cols.append(col)
        
print("Удалено:", len(drop_cols))
for drop_col in drop_cols:
    print(drop_col, X_train[drop_col].isna().mean(), test_df[drop_col].isna().mean())

X_train = X_train.drop(columns=drop_cols)
X_val   = X_val.drop(columns=drop_cols)
test_df = test_df.drop(columns=drop_cols)


Удалено: 8
id 0.0 0.0
платежный_график 0.0 0.0
коэфф_невыплаченного_сумм_остатка 0.0 0.0
непогашенная_сумма_из_тела_займов 0.0 0.0
пени_за_дефолт 0.3000186863206841 1.0
дата_следующей_выплаты 1.0 1.0
код_политики 0.0 0.0
особая_ситуация 0.0 0.0


### Категориальные поля

Выводим для каждого категориального столбца его
* Название
* Число уникальных значений
* Число пропусков
* Топ 5 самых частых значений

In [18]:
cat_cols = X_train.select_dtypes(exclude='number').columns
for cat_col in cat_cols:
    data = X_train[cat_col]
    print(cat_col)
    print('Уникальных значений', len(data.unique()))
    print(data.unique()[:5])
    print('Пропущенных значений', data.isna().sum())
    print(data.value_counts(ascending=False).head(5))
    print()
print(target, len(y_train.unique()))
print(y_train.unique())
print()

срок_займа
Уникальных значений 2
['5 лет' '3 года']
Пропущенных значений 0
срок_займа
3 года    734600
5 лет     234023
Name: count, dtype: int64

рейтинг
Уникальных значений 7
['В' 'Б' 'Д' 'А' 'Г']
Пропущенных значений 0
рейтинг
Б    282750
В    275200
А    169139
Г    144497
Д     67411
Name: count, dtype: int64

допрейтинг
Уникальных значений 35
['В1' 'Б1' 'Д2' 'А5' 'А4']
Пропущенных значений 0
допрейтинг
В1    61464
Б4    59989
Б5    59524
Б3    58833
В2    57202
Name: count, dtype: int64

профессия_заемщика
Уникальных значений 229882
['ответственный за бухгалтерский учет' 'федеральный агент'
 'медицинское обслуживание'
 'ит специалист по анализу оперативной деятельности' 'менеджер ит']
Пропущенных значений 61739
профессия_заемщика
менеджер         18678
преподаватель    17892
владелец         11232
медсестра        11178
водитель          9162
Name: count, dtype: int64

стаж
Уникальных значений 12
['10+ лет' '3 года' '2 года' '< 1 года' '7 лет']
Пропущенных значений 56482
стаж
10+

Заполним пропуски в категориальных столбцах, где это возможно и необходимо. Пропуски есть в полях:
* `профессия заемщика` - обработке поля посвящен отдельный раздел
* `стаж` -> варианта 2: заполняем либо `< 1 года`, либо особенным значением `unknown`. Лучше заполним `unknown`, т.к. иначе сильно исказим распределение значений.
* `совокупный_статус_подтверждения_доходов_заемщиков` - тут по смыслу `Не подтвержден`, т.к. у нас нет информации

In [19]:
for df in [X_train, X_val, test_df]:
    df['стаж'] = df['стаж'].fillna('unknown')
    df['совокупный_статус_подтверждения_доходов_заемщиков'] = df['совокупный_статус_подтверждения_доходов_заемщиков'].fillna('Не подтвержден')

Проверка

In [20]:
print(X_train['стаж'].value_counts())
print(X_train['совокупный_статус_подтверждения_доходов_заемщиков'].value_counts())

стаж
10+ лет     318552
2 года       87486
< 1 года     78087
3 года       77451
1 год        63755
5 лет        60407
4 года       57917
unknown      56482
6 лет        45115
8 лет        43768
7 лет        42930
9 лет        36673
Name: count, dtype: int64
совокупный_статус_подтверждения_доходов_заемщиков
Не подтвержден          960888
Подтвержден источник      4507
Подтвержден               3228
Name: count, dtype: int64


Большинство столбцов действительно категориальные, которые имеет смысл использовать для обучения модели, но есть исключения:
* `профессия заемщика` - почти 230000 уникальных значений, есть более и менее популярные, но в любом случае поле сложно использовать, т.к. каждая отдельная профессия встречается не более чем в 20000 записей из 1200000 (менее 2% для самой популярной профессии). Возможно закодировать только часть профессий, поскольку поле может оказаться важным для модели.
* `дата_первого_займа` - это дата, нужно привести к типу даты

In [21]:
display(X_train['профессия_заемщика'].value_counts().sample(20))

профессия_заемщика
средняя школа ассаты                                             1
внутренний консультант по продажам                               3
микробиология 3                                                  1
&quot; сколертек инк. &quot;                                     1
tvpm по восточному побережью                                     1
пищевые услуги &lt; &lt; содексо &gt; &gt;                       1
глобальное обслуживание по потоку                                4
сотрудник по вопросам поддержания мира (надзор)                  1
инженер по эксплуатации автотранспортных средств                 1
психолог-резервист                                               1
директор, процесс и портфель mgmt                                1
специалист по поддержке пк ii                                    1
акзо-нобелы краски                                               1
исполнительная власть во всем мире                               1
финантерпрактическая технология            

Можем видеть смешение разного написания профессий, ролей, отраслей, работодателей и мусора.
* синонимы:

    + менеджер, менеджер sr, менеджер магазина, менеджер операций

    + инженер, инженер сети, инженер проекта, инженер-механик

* разные языки:

    + manager, vp, sr, qa, it

    + русские переводы + английские оригиналы

* уровни должностей вместо профессий:

    + старший, вице-президент, директор

    + это роль, не профессия

* отрасли вместо профессий:

    + финансы

    + маркетинг

    + образование

* работодатели вместо профессий:

    + ibm

    + boeing

    + walmart

    + банк америки

* мусор:

    + html-артефакты

    + странные токены

    + обрезанные слова

    + опечатки

#### Работа с полем `профессия заемщика`

Нормализация профессий

In [22]:
# -----------------------------
# Расширенная нормализация профессий
# -----------------------------

eng_map = {
    "sr": "старший",
    "jr": "младший",
    "vp": "вице президент",
    "mgr": "менеджер",
    "qa": "качество",
    "it": "ит",
    "rn": "медсестра",
    "lpn": "медсестра",
    "cna": "медсестра",
    "driver": "водитель",
    "engineer": "инженер",
    "developer": "разработчик",
    "teacher": "учитель",
    "sales": "продажи",
}

def normalize_profession(s):
    if pd.isna(s):
        return "unknown"
    
    s = str(s).lower()
    
    # html / мусор
    s = re.sub(r'<.*?>', ' ', s)
    s = re.sub(r'[^a-zа-я0-9\s\-]', ' ', s)
    
    # замены англ сокращений
    for k, v in eng_map.items():
        s = re.sub(rf'\b{k}\b', v, s)
    
    s = re.sub(r'\s+', ' ', s).strip()
    
    if len(s) == 0:
        return "unknown"
        
    return s


for df in [X_train, X_val, test_df]:
    df['профессия_заемщика'] = df['профессия_заемщика'].apply(normalize_profession)


Выделение работодателей отдельно

In [23]:
# -----------------------------
# Детектор компаний
# -----------------------------

company_keywords = r"ibm|boeing|walmart|verizon|bank|банк|corp|inc|ltd|ooo|llc"

def is_company(s):
    return int(bool(re.search(company_keywords, s)))

for df in [X_train, X_val, test_df]:
    df["prof_is_company"] = df["профессия_заемщика"].apply(is_company)


Группы профессий

In [24]:
profession_groups = {
    "медицина": r"мед|врач|терапевт|стоматолог|фармацевт|медсестра|клиник",
    "образование": r"учител|преподавател|профессор|лектор",
    "it": r"программист|разработчик|данных|аналитик|qa|тестировщик|девопс",
    "инженерия": r"инженер|механик|техник|электрик",
    "финансы": r"бухгалтер|банкир|финанс|кредит|аудитор",
    "менеджмент": r"менеджер|директор|руководитель|вице президент",
    "продажи": r"продаж|торгов|sales",
    "маркетинг": r"маркетинг|реклама|pr",
    "юристы": r"адвокат|юрист|legal",
    "госслужба": r"гос|муницип|департамент",
    "силовые": r"полици|армия|военн|офицер|шериф",
    "транспорт": r"водител|перевоз|дальнобой",
    "строительство": r"строит|прораб|плотник",
    "производство": r"завод|фабрик|станок|оператор",
    "логистика": r"склад|кладовщик|погрузчик",
    "сервис": r"официант|бармен|кассир|повар|уборщик",
    "административные": r"секретар|администратор|клерк",
    "наука": r"scientist|исследователь",
    "бизнес": r"владелец|предприниматель|self employed"
}


Функция маппинга

In [25]:
def map_prof_group(s):
    if re.search(company_keywords, s):
        return "company_employee"
    
    for group, pattern in profession_groups.items():
        if re.search(pattern, s):
            return group
            
    return "other"


for df in [X_train, X_val, test_df]:
    df['prof_group'] = df['профессия_заемщика'].apply(map_prof_group)


Анализ редких профессий

In [26]:
freq = X_train['профессия_заемщика'].value_counts()

RARE_THRESHOLD = 50
rare_prof = set(freq[freq < RARE_THRESHOLD].index)

def mark_rare(s, g):
    if s in rare_prof and g == "other":
        return "rare_other"
    return s

for df in [X_train, X_val, test_df]:
    df['prof_clean'] = df.apply(
        lambda r: mark_rare(r['профессия_заемщика'], r['prof_group']),
        axis=1
    )


Частотный encoding профессии

In [27]:
prof_freq_map = X_train['prof_clean'].value_counts(normalize=True)

for df in [X_train, X_val, test_df]:
    df['prof_freq_enc'] = df['prof_clean'].map(prof_freq_map).fillna(0)


In [ ]:
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np


def kfold_target_encoding(
    train_col,
    y,
    test_col,
    n_splits=5,
    smoothing=20,
    noise=0.0,
    random_state=42
):
    """
    KFold Target Encoding с защитой от утечки целевого признака
    
    train_col — категориальный признак train (pd.Series)
    y         — таргет train (pd.Series)
    test_col  — тот же признак в val/test
    
    smoothing — сила сглаживания
    noise     — добавление шума для регуляризации
    """

    train_col = train_col.astype(str)
    test_col  = test_col.astype(str)

    global_mean = y.mean()

    kf = StratifiedKFold(
        n_splits=n_splits,
        shuffle=True,
        random_state=random_state
    )

    train_encoded = pd.Series(index=train_col.index, dtype=float)

    # ---------- OOF encoding ----------
    for tr_idx, val_idx in kf.split(train_col, y):

        tr_cat = train_col.iloc[tr_idx]
        tr_y   = y.iloc[tr_idx]

        stats = tr_y.groupby(tr_cat).agg(['mean', 'count'])

        smooth = (
            stats['mean'] * stats['count'] +
            global_mean * smoothing
        ) / (stats['count'] + smoothing)

        val_map = train_col.iloc[val_idx].map(smooth)

        train_encoded.iloc[val_idx] = val_map.fillna(global_mean)

    # сглаживание
    stats_full = y.groupby(train_col).agg(['mean', 'count'])

    smooth_full = (
        stats_full['mean'] * stats_full['count'] +
        global_mean * smoothing
    ) / (stats_full['count'] + smoothing)

    test_encoded = test_col.map(smooth_full).fillna(global_mean)

    # добавление шума
    if noise > 0:
        train_encoded *= (1 + noise * np.random.randn(len(train_encoded)))
        test_encoded  *= (1 + noise * np.random.randn(len(test_encoded)))

    return train_encoded, test_encoded


Частотный encoding по группе

In [29]:
group_freq_map = X_train['prof_group'].value_counts(normalize=True)

for df in [X_train, X_val, test_df]:
    df['prof_group_freq'] = df['prof_group'].map(group_freq_map)


Target encoding по группе профессии

In [30]:

X_train['prof_group_te'], X_val['prof_group_te'] = kfold_target_encoding(
    X_train['prof_group'],
    y_train,
    X_val['prof_group'],
    smoothing=50,
    noise=0.01
)

_, test_df['prof_group_te'] = kfold_target_encoding(
    X_train['prof_group'],
    y_train,
    test_df['prof_group']
)


Target encoding по профессии

In [31]:
X_train['prof_te'], X_val['prof_te'] = kfold_target_encoding(
    X_train['prof_clean'],
    y_train,
    X_val['prof_clean'],
    smoothing=50,
    noise=0.01
)

_, test_df['prof_te'] = kfold_target_encoding(
    X_train['prof_clean'],
    y_train,
    test_df['prof_clean'],
    smoothing=50
)


Удаление промежуточных полей

In [ ]:
drop_prof_cols = [
    'prof_clean'
]

X_train = X_train.drop(columns=drop_prof_cols)
X_val   = X_val.drop(columns=drop_prof_cols)
test_df = test_df.drop(columns=drop_prof_cols)


Поле `профессия заемщика` удалим позже

В итоге получили:
* `prof_group`           ← отрасль
* `prof_group_te`        ← риск отрасли
* `prof_group_freq`      ← размер отрасли
* `prof_te`              ← риск профессии
* `prof_freq_enc`        ← частота профессии
* `prof_is_company`      ← флаг работодателя

Обновление списка категориальных полей

In [33]:
cat_cols = X_train.select_dtypes(exclude='number').columns

Проверка новых полей

In [34]:
print('Средняя вероятность дефолта =', y_train.mean())
display(X_train[['prof_group', 'prof_group_te', 'prof_group_freq',\
    'prof_te', 'prof_freq_enc', 'prof_is_company']].head(15))

Средняя вероятность дефолта = 0.19961842739641739


,prof_group,prof_group_te,prof_group_freq,prof_te,prof_freq_enc,prof_is_company
938963,финансы,0.171955,0.025341,0.215943,0.000147,0
108797,other,0.200495,0.537837,0.143150,0.000063,0
287505,медицина,0.196439,0.045688,0.256609,0.000142,0
403826,other,0.203392,0.537837,0.155769,0.000067,0
484031,менеджмент,0.190953,0.154114,0.124506,0.001251,0
85910,медицина,0.195799,0.045688,0.196746,0.000231,0
613914,other,0.202896,0.537837,0.207169,0.000104,0
325319,транспорт,0.261510,0.027912,0.289613,0.000817,0
1077974,менеджмент,0.191226,0.154114,0.154490,0.000041,0
692939,other,0.203503,0.537837,0.188011,0.278487,0


Поля добавлены, средние групповые значения не слишком отличаются от среднего по всем данным, но все равно могут увеличить точности модели

In [35]:
X_train['prof_group'].value_counts(normalize=True)


prof_group
other               0.537837
менеджмент          0.154114
инженерия           0.050426
медицина            0.045688
образование         0.030509
транспорт           0.027912
финансы             0.025341
продажи             0.022925
it                  0.016442
бизнес              0.014539
производство        0.013670
company_employee    0.013281
административные    0.011498
силовые             0.010109
сервис              0.007085
госслужба           0.006444
юристы              0.003957
маркетинг           0.003386
строительство       0.002179
логистика           0.002152
наука               0.000506
Name: proportion, dtype: float64

Больше половины значений не причислилось ни к одной из категорий, но влияение на метрику в любом случае нужно проверять

In [36]:
X_train.loc[X_train.prof_group == "other", "профессия_заемщика"] \
    .value_counts() \
    .head(50)

профессия_заемщика
unknown                                           61744
рн                                                 6869
помощник по административным вопросам              5855
генеральный управляющий                            3974
президент                                          3676
вице-президент                                     3325
управляющий счетом                                 2531
форман                                             2183
сервер                                             2164
технический сотрудник                              1775
консультант                                        1711
параюридический                                    1678
техническое обслуживание                           1652
служба обслуживания клиентов                       1621
контролер                                          1613
исполнительный счет                                1561
суперинтендант                                     1502
сотрудник исправительных учре

In [37]:
X_train.drop('профессия_заемщика', axis=1, inplace=True)

Теперь вместо профессии имеем несколько полей, исходное поле удалено

Изменение типа поля `дата_первого_займа`

Вместо одного поля с датой имеем 2 поля - год и месяц

In [38]:
for df in [X_train, X_val, test_df]:
    df['дата_первого_займа'] = pd.to_datetime(
        df['дата_первого_займа'],
        format='%m-%Y',
        errors='coerce'
    )

    df['first_loan_year']  = df['дата_первого_займа'].dt.year
    df['first_loan_month'] = df['дата_первого_займа'].dt.month

    df.drop(columns=['дата_первого_займа'], inplace=True)

Добавление флагов для полей с пропусками

In [39]:
high_missing = missing_train[missing_train > 0.2].index

def add_missing_flags(df, cols):
    df = df.copy()
    for c in cols:
        if c in df:
            df[c+"_miss"] = df[c].isna().astype(int)
    return df

X_train = add_missing_flags(X_train, high_missing)
X_val   = add_missing_flags(X_val, high_missing)
test_df = add_missing_flags(test_df, high_missing)


### Непрерывные поля

Заполнение числовых полей по смыслу:
* '%месяц%' -> 999
* '%кол-во%' -> 0
* остальные -> медиана

In [40]:
month_cols = [c for c in X_train.columns if "месяц" in c]

for c in month_cols:
    X_train[c] = X_train[c].fillna(999)
    X_val[c]   = X_val[c].fillna(999)
    test_df[c] = test_df[c].fillna(999)

In [41]:
count_cols = [c for c in X_train.columns if "кол-во" in c]

for c in count_cols:
    X_train[c] = X_train[c].fillna(0)
    X_val[c]   = X_val[c].fillna(0)
    test_df[c] = test_df[c].fillna(0)


In [42]:
num_cols = X_train.select_dtypes(include='number').columns
cat_cols = X_train.select_dtypes(exclude='number').columns
med = X_train[num_cols].median()

X_train[num_cols] = X_train[num_cols].fillna(med)
X_val[num_cols]   = X_val[num_cols].fillna(med)
test_df[num_cols] = test_df[num_cols].fillna(med)


Далее будут выбраны признаки для модели с наибольшими `Information Value`

Функция для расчета Information Value

In [43]:
def calc_iv(df, feature, target, bins=10):
    d = df[[feature, target]].copy()
    
    try:
        if d[feature].nunique() > bins:
            d["bin"] = pd.qcut(d[feature], bins, duplicates='drop')
        else:
            d["bin"] = d[feature]
    except:
        return 0
    
    g = d.groupby("bin")[target].agg(["count","sum"])
    g.columns = ["total","bad"]
    g["good"] = g.total - g.bad
    
    g["bad_rate"]  = g.bad / g.bad.sum()
    g["good_rate"] = g.good / g.good.sum()
    
    g["woe"] = np.log((g.good_rate+1e-6)/(g.bad_rate+1e-6))
    g["iv"]  = (g.good_rate-g.bad_rate)*g.woe
    
    return g.iv.sum()


Вычисляем IV для числовых признаков

In [44]:
tmp = X_train.copy()
tmp[target] = y_train

iv_scores = {}

for col in X_train.columns:
    if col in cat_cols:
        continue
    iv_scores[col] = calc_iv(tmp, col, target)

iv_series = pd.Series(iv_scores).sort_values(ascending=False);
display(iv_series.head(30))


C:\Temp\ipykernel_19292\3531124842.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = d.groupby("bin")[target].agg(["count","sum"])
C:\Temp\ipykernel_19292\3531124842.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = d.groupby("bin")[target].agg(["count","sum"])
C:\Temp\ipykernel_19292\3531124842.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = d.groupby("bin")[target].agg(["count","sum"])
C:\Temp\ip

процентная_ставка                                    0.444614
сумма_выплат_по_просрочкам                           0.211605
верхний_порог_рейтинга_заемщика                      0.122058
нижний_порог_рейтинга_заемщика                       0.122058
пдн                                                  0.072216
кол-во_открытых_счетов_за_2_года                     0.066176
суммарная_доступная_сумма_займа_по_картам            0.053564
средний_баланс_текущих_счетов                        0.046855
кол-во_счетов_за_посл_год                            0.046131
prof_te                                              0.044260
кредитный_лимит                                      0.042718
лимит_по_картам                                      0.036020
общая_сумма_на_счетах                                0.035654
сумма_займа                                          0.034848
кол-во_активных_возобновляемых_счетов                0.034826
кол-во_возобновляемых_счетов_с_балансом_более_0      0.033318
кол-во_м

Отбор по IV

In [45]:
iv_selected = iv_series[iv_series > 0.01].index.tolist()
print("IV selected:", len(iv_selected))


IV selected: 56


Отбор по корреляции. Избавляемся от мультиколлинеарности

In [46]:
corr = X_train[iv_selected].corr().abs()
# только верхний треугольник матрицы корреляции
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

to_drop_corr = [
    column for column in upper.columns
    if any(upper[column] > 0.9)
]

final_features = [c for c in iv_selected if c not in to_drop_corr]

print("После корреляции:", len(final_features))


После корреляции: 37


In [47]:
print(final_features)

['процентная_ставка', 'сумма_выплат_по_просрочкам', 'верхний_порог_рейтинга_заемщика', 'пдн', 'кол-во_открытых_счетов_за_2_года', 'суммарная_доступная_сумма_займа_по_картам', 'средний_баланс_текущих_счетов', 'кол-во_счетов_за_посл_год', 'prof_te', 'кредитный_лимит', 'лимит_по_картам', 'сумма_займа', 'кол-во_активных_возобновляемых_счетов', 'кол-во_месяцев_с_последнего_счета', 'кол-во_ипотек', 'соотношение_баланса_к_лимиту_общее', 'кол-во_возоб_счетов_за_2_года', 'процент_счетов_прев_75_лимита', 'соотношение_баланса_к_лимиту_по_картам', 'годовой_доход', 'кол-во_месяцев_с_последней_карты', 'общий_лимит_по_возоб_счету', 'кол-во_заявок_за_полгода', 'кол-во_месяцев_с_первого_возобновляемого_счета', 'коэфф_загрузки_возобновляемого_счета', 'кол-во_возоб_счетов_за_год', 'кол-во_заявок_на_кредит_за_год', 'соотношение_сумм_текущего_баланса_к_лимиту_по_аннуитетным_счетам', 'first_loan_year', 'кол-во_месяцев_с_посл_аннуитетного_счета', 'кол-во_открытых_счетов_за_полгода', 'кол-во_активных_карт', '

Вспомогательные списки

In [48]:
prof_num = [
    'prof_group_te',
    'prof_group_freq',
    'prof_te',
    'prof_freq_enc',
    'prof_is_company'
]

prof_cat = ['prof_group']

# Проверка наличия
for c in prof_num + prof_cat:
    assert c in X_train.columns, f"{c} not in X_train"


IV для этих полей

In [49]:
tmp = X_train.copy()
tmp[target] = y_train

iv_scores = {}

for col in prof_num:
    iv_scores[col] = calc_iv(tmp, col, target)
    
iv_df = pd.DataFrame(list(iv_scores.values()), index = prof_num, columns=['IV'])
display(iv_df)

C:\Temp\ipykernel_19292\3531124842.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = d.groupby("bin")[target].agg(["count","sum"])
C:\Temp\ipykernel_19292\3531124842.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = d.groupby("bin")[target].agg(["count","sum"])
C:\Temp\ipykernel_19292\3531124842.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = d.groupby("bin")[target].agg(["count","sum"])
C:\Temp\ip

,IV
prof_group_te,0.013180
prof_group_freq,0.001686
prof_te,0.044260
prof_freq_enc,0.010135
prof_is_company,0.001191


Проверка типов полей для профессии

In [50]:
X_train[prof_num + prof_cat].info()

<class 'pandas.core.frame.DataFrame'>
Index: 968623 entries, 938963 to 636513
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   prof_group_te    968623 non-null  float64
 1   prof_group_freq  968623 non-null  float64
 2   prof_te          968623 non-null  float64
 3   prof_freq_enc    968623 non-null  float64
 4   prof_is_company  968623 non-null  int64  
 5   prof_group       968623 non-null  object 
dtypes: float64(4), int64(1), object(1)
memory usage: 51.7+ MB


Типы верные

In [51]:
X_train[X_train.select_dtypes(exclude='number').columns].info()

<class 'pandas.core.frame.DataFrame'>
Index: 968623 entries, 938963 to 636513
Data columns (total 15 columns):
 #   Column                                             Non-Null Count   Dtype 
---  ------                                             --------------   ----- 
 0   срок_займа                                         968623 non-null  object
 1   рейтинг                                            968623 non-null  object
 2   допрейтинг                                         968623 non-null  object
 3   стаж                                               968623 non-null  object
 4   владение_жильем                                    968623 non-null  object
 5   подтвержден_ли_доход                               968623 non-null  object
 6   цель_займа                                         968623 non-null  object
 7   регион                                             968623 non-null  object
 8   пос_стоп_фактор                                    968623 non-null  object
 9   юрид

In [52]:
X_train[final_features].info()

<class 'pandas.core.frame.DataFrame'>
Index: 968623 entries, 938963 to 636513
Data columns (total 37 columns):
 #   Column                                                            Non-Null Count   Dtype  
---  ------                                                            --------------   -----  
 0   процентная_ставка                                                 968623 non-null  float64
 1   сумма_выплат_по_просрочкам                                        968623 non-null  float64
 2   верхний_порог_рейтинга_заемщика                                   968623 non-null  float64
 3   пдн                                                               968623 non-null  float64
 4   кол-во_открытых_счетов_за_2_года                                  968623 non-null  float64
 5   суммарная_доступная_сумма_займа_по_картам                         968623 non-null  float64
 6   средний_баланс_текущих_счетов                                     968623 non-null  float64
 7   кол-во_счетов_за_пос

Формируем наборы признаков

* Числовые с высоким IV без профессии + все категориальные без профессии
* Числовые с высоким IV с профессией (полученные признаки) + все категориальные с категорией профессии
* Все с высоким IV, 3 числовых для профессии + все категориальные с категорией профессии

In [53]:
cat_cols = X_train.select_dtypes(exclude='number').columns

# A — без профессии
features_A_num = [feature for feature in final_features if feature not in prof_num]
features_A_cat = [c for c in cat_cols if c not in prof_cat]
print('Пересечение признаков A =',set(features_A_num) & set(features_A_cat))
features_A = []
for fAn in features_A_num:
    features_A.append(fAn)
for fAc in features_A_cat:
    features_A.append(fAc)

# B — с профессией (полный)
features_B_num = final_features + [col for col in prof_num if col not in final_features]
features_B_cat = list(cat_cols.copy())
print('Пересечение признаков B =',set(features_B_num) & set(features_B_cat))
features_B = []
for fBn in features_B_num:
    features_B.append(fBn)
for fBc in features_B_cat:
    features_B.append(fBc)

# C — минимальный набор профессии
features_C_num = final_features + [col for col in [
    'prof_group_te',
    'prof_te',
    'prof_freq_enc'
] if col not in final_features]
features_C_cat = ['prof_group']
print('Пересечение признаков C =',set(features_C_num) & set(features_C_cat))

print(len(features_A_cat), len(features_A_num), len(features_A_cat)+len(features_A_num))
print(len(features_B_cat), len(features_B_num), len(features_B_cat)+len(features_B_num))
print(len(features_C_cat), len(features_C_num), len(features_C_cat)+len(features_C_num))


Пересечение признаков A = set()
Пересечение признаков B = set()
Пересечение признаков C = set()
14 34 48
15 39 54
1 37 38


Проверка на наличие пропусков

In [54]:
print('Пропуски в наборе признаков B')
display(X_train[features_B]\
    .isna().sum().sort_values(ascending=False).head())

Пропуски в наборе признаков B


процентная_ставка                           0
рейтинг                                     0
кол-во_месяцев_с_посл_аннуитетного_счета    0
кол-во_открытых_счетов_за_полгода           0
кол-во_активных_карт                        0
dtype: int64

Отлично, пропусков нет

### Проверка наличия явных дупликатов

In [55]:
X = pd.concat([X_train, X_val], axis=0)
y = pd.concat([y_train, y_val], axis=0)
display(X.duplicated().sum())

0

Явные дупликаты отсутствуют

Функция для обучения CatBoost

Пока что используем `CatBoost` как наиболее универсальную

In [56]:
def train_cb_cv(
    X,
    y,
    num_feats,
    cat_feats,
    n_splits=5,
    random_state=42,
    name="",
    return_models=False
):
    features = num_feats + cat_feats

    skf = StratifiedKFold(
        n_splits=n_splits,
        shuffle=True,
        random_state=random_state
    )

    oof_pred = np.zeros(len(X))
    aucs = []
    models = []

    for fold, (tr_idx, val_idx) in enumerate(skf.split(X, y), 1):

        X_tr, X_va = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[val_idx]

        model = CatBoostClassifier(
            iterations=300,
            learning_rate=0.1,
            depth=6,
            l2_leaf_reg=5,
            loss_function='Logloss',
            eval_metric='AUC',
            random_seed=random_state,
            verbose=False,
            task_type="GPU",
            devices='0',
            early_stopping_rounds=50
        )

        model.fit(
            X_tr[features],
            y_tr,
            eval_set=(X_va[features], y_va),
            cat_features=cat_feats,
            use_best_model=True
        )

        val_pred = model.predict_proba(X_va[features])[:, 1]
        oof_pred[val_idx] = val_pred

        fold_auc = roc_auc_score(y_va, val_pred)
        aucs.append(fold_auc)

        print(f"Fold {fold}: AUC = {fold_auc:.5f}")

        if return_models:
            models.append(model)

    mean_auc = roc_auc_score(y, oof_pred)

    print(f"\n{name} CV AUC = {mean_auc:.5f}")
    print(f"Fold AUCs: {[round(a, 5) for a in aucs]}")

    if return_models:
        return models, mean_auc, oof_pred
    else:
        return mean_auc, oof_pred

Обучение моделей

Модель A

In [ ]:
auc_A, _ = train_cb_cv(
    X_train,
    y_train,
    features_A_num,
    features_A_cat,
    name="A: no profession"
)


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 1: AUC = 0.75470


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 2: AUC = 0.75472


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 3: AUC = 0.75603


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 4: AUC = 0.75498


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 5: AUC = 0.75528

A: no profession CV AUC = 0.75514
Fold AUCs: [0.7547, 0.75472, 0.75603, 0.75498, 0.75528]


Модель B

In [ ]:
auc_B, _ = train_cb_cv(
    X_train,
    y_train,
    features_B_num,
    features_B_cat,
    name="B: full profession"
)

Default metric period is 5 because AUC is/are not implemented for GPU


Fold 1: AUC = 0.75754


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 2: AUC = 0.75671


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 3: AUC = 0.75800


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 4: AUC = 0.75745


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 5: AUC = 0.75780

B: full profession CV AUC = 0.75750
Fold AUCs: [0.75754, 0.75671, 0.758, 0.75745, 0.7578]


Модель C

In [ ]:
auc_C, _ = train_cb_cv(
    X_train,
    y_train,
    features_C_num,
    features_C_cat,
    name="C: minimal profession"
)

Default metric period is 5 because AUC is/are not implemented for GPU


Fold 1: AUC = 0.74643


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 2: AUC = 0.74677


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 3: AUC = 0.74747


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 4: AUC = 0.74687


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 5: AUC = 0.74764

C: minimal profession CV AUC = 0.74703
Fold AUCs: [0.74643, 0.74677, 0.74747, 0.74687, 0.74764]


### Выбор признаков для модели

Модель на всех признаках для отбора

Все признаки для отбора - это лучшие по IV после удаления высоко скоррелированных, а также все категориальные

In [57]:
final_num_features = final_features.copy()
final_cat_features = list(X_train.select_dtypes(exclude='number').columns)

In [58]:
full_num_features = final_num_features.copy()
full_cat_features = final_cat_features.copy()

full_features = full_num_features + full_cat_features

print("Числовых:", len(full_num_features))
print("Категориальных:", len(full_cat_features))
print("Всего:", len(full_features))


Числовых: 37
Категориальных: 15
Всего: 52


In [62]:
auc_full, _ = train_cb_cv(
    X_train, y_train,
    full_num_features,
    full_cat_features,
    name="FULL MODEL (num + cat)"
)

Default metric period is 5 because AUC is/are not implemented for GPU


Fold 1: AUC = 0.75744


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 2: AUC = 0.75688


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 3: AUC = 0.75798


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 4: AUC = 0.75720


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 5: AUC = 0.75793

FULL MODEL (num + cat) CV AUC = 0.75748
Fold AUCs: [0.75744, 0.75688, 0.75798, 0.7572, 0.75793]


In [65]:
RANDOM_STATE = 42
model_full = model = CatBoostClassifier(
            iterations=300,
            learning_rate=0.1,
            depth=6,
            l2_leaf_reg=5,
            loss_function='Logloss',
            eval_metric='AUC',
            random_seed=RANDOM_STATE,
            verbose=False,
            task_type="GPU",
            devices='0',
            early_stopping_rounds=50
        )
model_full.fit(
            X_train[full_num_features+full_cat_features],
            y_train,
            cat_features=full_cat_features
        )

Default metric period is 5 because AUC is/are not implemented for GPU


Важности признаков `CatBoost`

In [66]:
importances = model_full.get_feature_importance()
feat_imp = pd.Series(importances, index=full_features).sort_values(ascending=False)

display(feat_imp.head(20))


процентная_ставка                                 23.434250
сумма_выплат_по_просрочкам                        17.572750
срок_займа                                         7.967669
prof_te                                            4.555327
сумма_займа                                        4.424109
пдн                                                3.735464
верхний_порог_рейтинга_заемщика                    2.927681
кол-во_открытых_счетов_за_2_года                   2.845274
годовой_доход                                      2.403613
владение_жильем                                    2.046521
кредитный_лимит                                    1.975821
средний_баланс_текущих_счетов                      1.901887
кол-во_активных_возобновляемых_счетов              1.729332
рейтинг                                            1.696723
кол-во_месяцев_с_посл_аннуитетного_счета           1.587397
кол-во_ипотек                                      1.567994
регион                                  

Тест фиксированного числа признаков из топа

In [69]:
def split_num_cat(feature_list):
    num = [f for f in feature_list if f in full_num_features]
    cat = [f for f in feature_list if f in full_cat_features]
    return num, cat


def evaluate_top_k_importance(k):
    selected = feat_imp.index[:k].tolist()
    num_sel, cat_sel = split_num_cat(selected)
    
    auc,_ = train_cb_cv(
        X_train, y_train,
        num_sel,
        cat_sel,
        name=f"CatBoost importance top {k}"
    )
    
    model = CatBoostClassifier(
                iterations=300,
                learning_rate=0.1,
                depth=6,
                l2_leaf_reg=5,
                loss_function='Logloss',
                eval_metric='AUC',
                random_seed=RANDOM_STATE,
                verbose=False,
                task_type="GPU",
                devices='0',
                early_stopping_rounds=50
            )
    model.fit(
                X_train[num_sel+cat_sel],
                y_train,
                cat_features=cat_sel
            )
    
    return model, auc, selected


results_imp1 = {}

for k in [15, 25, 40, len(full_features)]:
    model_k, auc_k, feats_k = evaluate_top_k_importance(k)
    results_imp1[k] = (model_k, auc_k, feats_k)

best_imp = max(results_imp1.items(), key=lambda x: x[1][1])
print("Best CatBoost importance:", best_imp[0], "AUC:", best_imp[1][1])


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 1: AUC = 0.74951


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 2: AUC = 0.74903


Default metric period is 5 because AUC is/are not implemented for GPU


...

Fold 4: AUC = 0.75725


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 5: AUC = 0.75799

CatBoost importance top 52 CV AUC = 0.75749
Fold AUCs: [0.75749, 0.75685, 0.75788, 0.75725, 0.75799]


Default metric period is 5 because AUC is/are not implemented for GPU


Best CatBoost importance: 52 AUC: 0.7574910029582489


In [70]:

for k in [30, 35, 45, 50]:
    model_k, auc_k, feats_k = evaluate_top_k_importance(k)
    results_imp1[k] = (model_k, auc_k, feats_k)

best_imp = max(results_imp1.items(), key=lambda x: x[1][1])
print("Best CatBoost importance:", best_imp[0], "AUC:", best_imp[1][1])

Default metric period is 5 because AUC is/are not implemented for GPU


Fold 1: AUC = 0.75666


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 2: AUC = 0.75564


Default metric period is 5 because AUC is/are not implemented for GPU


...

Fold 4: AUC = 0.75737


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 5: AUC = 0.75786

CatBoost importance top 50 CV AUC = 0.75746
Fold AUCs: [0.75746, 0.75663, 0.75798, 0.75737, 0.75786]


Default metric period is 5 because AUC is/are not implemented for GPU


Best CatBoost importance: 45 AUC: 0.757526724069344


In [72]:

for k in [36, 38, 42, 44, 46, 48]:
    model_k, auc_k, feats_k = evaluate_top_k_importance(k)
    results_imp1[k] = (model_k, auc_k, feats_k)

best_imp = max(results_imp1.items(), key=lambda x: x[1][1])
print("Best CatBoost importance:", best_imp[0], "AUC:", best_imp[1][1])

Default metric period is 5 because AUC is/are not implemented for GPU


Fold 1: AUC = 0.75728


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 2: AUC = 0.75661


Default metric period is 5 because AUC is/are not implemented for GPU


...

Fold 4: AUC = 0.75727


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 5: AUC = 0.75785

CatBoost importance top 48 CV AUC = 0.75746
Fold AUCs: [0.75749, 0.75673, 0.75799, 0.75727, 0.75785]


Default metric period is 5 because AUC is/are not implemented for GPU


Best CatBoost importance: 42 AUC: 0.7575484112196919


Наибольшую метрику на кросс валидации получили на 39 признаках

Важности признаков SHAP

In [73]:
explainer = shap.TreeExplainer(model_full)

shap_values = explainer.shap_values(X_train[full_features])

shap_importance = np.abs(shap_values).mean(axis=0)

shap_imp_series = pd.Series(
    shap_importance,
    index=full_features
).sort_values(ascending=False)

display(shap_imp_series.head(20))


срок_займа                                        0.250196
процентная_ставка                                 0.242309
сумма_выплат_по_просрочкам                        0.152667
сумма_займа                                       0.133090
prof_te                                           0.123335
пдн                                               0.122913
кол-во_открытых_счетов_за_2_года                  0.097223
владение_жильем                                   0.091904
годовой_доход                                     0.083983
верхний_порог_рейтинга_заемщика                   0.076995
рейтинг                                           0.068681
кол-во_активных_возобновляемых_счетов             0.066395
регион                                            0.062277
кол-во_ипотек                                     0.054081
кол-во_возоб_счетов_за_2_года                     0.048082
кол-во_месяцев_с_первого_возобновляемого_счета    0.048040
кредитный_лимит                                   0.0464

Фиксированные число признаков из топа SHAP

In [ ]:
def evaluate_top_k_shap(k):
    selected = shap_imp_series.index[:k].tolist()
    num_sel, cat_sel = split_num_cat(selected)
    
    model, auc = train_cb_cv(
        X_train, y_train,
        num_sel,
        cat_sel,
        name=f"SHAP top {k}"
    )
    
    model = CatBoostClassifier(
                iterations=300,
                learning_rate=0.1,
                depth=6,
                l2_leaf_reg=5,
                loss_function='Logloss',
                eval_metric='AUC',
                random_seed=RANDOM_STATE,
                verbose=False,
                task_type="GPU",
                devices='0',
                early_stopping_rounds=50
            )
    model.fit(
                X_train[num_sel+cat_sel],
                y_train,
                cat_features=cat_sel
            )
    
    return model, auc, selected
    
    return model, auc, selected


results_shap = {}

for k in [15, 25, 40, len(full_features)]:
    model_k, auc_k, feats_k = evaluate_top_k_shap(k)
    results_shap[k] = (model_k, auc_k, feats_k)




Default metric period is 5 because AUC is/are not implemented for GPU


Fold 1: AUC = 0.75116


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 2: AUC = 0.75073


Default metric period is 5 because AUC is/are not implemented for GPU


...

Fold 4: AUC = 0.75722


Default metric period is 5 because AUC is/are not implemented for GPU


Fold 5: AUC = 0.75786

SHAP top 52 CV AUC = 0.75752
Fold AUCs: [0.75754, 0.75696, 0.75802, 0.75722, 0.75786]


Default metric period is 5 because AUC is/are not implemented for GPU


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

Не будем рассматривать конкретные числа признаков для SHAP и примем 42 признака из `CatBoost` за некий оптимум

Теперь перейдем к выбору модели

In [75]:
RANDOM_STATE = 42

cv = StratifiedKFold(
    n_splits=3,
    shuffle=True,
    random_state=RANDOM_STATE
)

Финальный набор признаков

In [76]:
features_to_model = feat_imp.head(42).index.tolist()

print("Используем признаков:", len(features_to_model))
features_to_model


Используем признаков: 42


['процентная_ставка',
 'сумма_выплат_по_просрочкам',
 'срок_займа',
 'prof_te',
 'сумма_займа',
 'пдн',
 'верхний_порог_рейтинга_заемщика',
 'кол-во_открытых_счетов_за_2_года',
 'годовой_доход',
 'владение_жильем',
 'кредитный_лимит',
 'средний_баланс_текущих_счетов',
 'кол-во_активных_возобновляемых_счетов',
 'рейтинг',
 'кол-во_месяцев_с_посл_аннуитетного_счета',
 'кол-во_ипотек',
 'регион',
 'кол-во_месяцев_с_первого_возобновляемого_счета',
 'кол-во_возоб_счетов_за_2_года',
 'общий_лимит_по_возоб_счету',
 'допрейтинг',
 'соотношение_баланса_к_лимиту_общее',
 'цель_займа',
 'лимит_по_картам',
 'кол-во_месяцев_с_последней_карты',
 'кол-во_заявок_за_полгода',
 'стаж',
 'first_loan_year',
 'prof_group_te',
 'кол-во_заявок_на_кредит_за_год',
 'соотношение_баланса_к_лимиту_по_картам',
 'процент_счетов_прев_75_лимита',
 'подтвержден_ли_доход',
 'коэфф_загрузки_возобновляемого_счета',
 'кол-во_активных_карт',
 'суммарная_доступная_сумма_займа_по_картам',
 'prof_freq_enc',
 'тип_займа',
 'ко

In [77]:
num_cols_model = [
    c for c in features_to_model
    if c in X_train.select_dtypes(include='number').columns
]

cat_cols_model = [
    c for c in features_to_model
    if c in X_train.select_dtypes(exclude='number').columns
]

print("Числовые:", len(num_cols_model))
print("Категориальные:", len(cat_cols_model))


Числовые: 33
Категориальные: 9


Препроцессинг для бустинга

In [78]:
numeric_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

categorical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent'))
])

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_pipe, num_cols_model),
        ('cat', categorical_pipe, cat_cols_model)
    ],
    remainder='drop'
)


Препроцессинг для LogReg

In [79]:
numeric_pipe_lr = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocess_lr = ColumnTransformer(
    transformers=[
        ('num', numeric_pipe_lr, num_cols_model)
    ],
    remainder='drop'
)


Индексы категориальных полей

In [83]:
cat_feature_indices = [
    X_train[features_to_model].columns.get_loc(col)
    for col in cat_cols_model
]

cat_feature_indices


[2, 9, 13, 16, 20, 22, 26, 32, 37]

Гиперпараметры моделей

In [84]:
model_spaces = [

    
    # Logistic Regression
    (
        "LogReg",
        Pipeline([
            ('prep', preprocess_lr),
            ('model', LogisticRegression(
                solver='saga',
                penalty='l1',
                max_iter=3000,
                class_weight='balanced',
                random_state=RANDOM_STATE,
                n_jobs=-1
            ))
        ]),
        {
            'model__C': distributions.FloatDistribution(1e-4, 1e2, log=True)
        }
    ),


    # CatBoost
    (
        "CatBoost",
        Pipeline([
            ('model', CatBoostClassifier(
                loss_function='Logloss',
                eval_metric='AUC',
                iterations=400,
                auto_class_weights='Balanced',
                random_seed=RANDOM_STATE,
                verbose=False,
                task_type='GPU',
                devices='0',
                cat_features=cat_feature_indices
            ))
        ]),
        {
            'model__depth': distributions.IntDistribution(4, 10),
            'model__learning_rate': distributions.FloatDistribution(0.02, 0.2, log=True),
            'model__l2_leaf_reg': distributions.FloatDistribution(1, 50, log=True),
            'model__bagging_temperature': distributions.FloatDistribution(0, 1),
            'model__random_strength': distributions.FloatDistribution(0, 2)
        }
    ),


    # LightGBM
    (
        "LightGBM",
        Pipeline([
            ('prep', preprocess),
            ('model', LGBMClassifier(
                objective='binary',
                n_estimators=500,
                random_state=RANDOM_STATE,
                n_jobs=-1,
                is_unbalance=True
            ))
        ]),
        {
            'model__num_leaves': distributions.IntDistribution(16, 64),
            'model__max_depth': distributions.IntDistribution(4, 10),
            'model__learning_rate': distributions.FloatDistribution(0.02, 0.2, log=True),
            'model__min_child_samples': distributions.IntDistribution(20, 100),
            'model__subsample': distributions.FloatDistribution(0.7, 1.0),
            'model__colsample_bytree': distributions.FloatDistribution(0.7, 1.0)
        }
    ),


    # XGBoost
    (
        "XGBoost",
        Pipeline([
            ('prep', preprocess),
            ('model', XGBClassifier(
                objective='binary:logistic',
                eval_metric='auc',
                n_estimators=500,
                tree_method='gpu_hist',
                predictor='gpu_predictor',
                random_state=RANDOM_STATE,
                n_jobs=-1
            ))
        ]),
        {
            'model__max_depth': distributions.IntDistribution(3, 8),
            'model__learning_rate': distributions.FloatDistribution(0.01, 0.3, log=True),
            'model__min_child_weight': distributions.IntDistribution(1, 20),
            'model__subsample': distributions.FloatDistribution(0.6, 1.0),
            'model__colsample_bytree': distributions.FloatDistribution(0.6, 1.0),
            'model__gamma': distributions.FloatDistribution(1e-3, 10, log=True),
            'model__reg_lambda': distributions.FloatDistribution(1e-3, 10, log=True)
        }
    )
]


Поиск лучших параметров

In [81]:
n_trials_dict = {
    'LogReg': 10,
    'CatBoost': 30,
    'LightGBM': 25,
    'XGBoost': 25
}

results = {}


In [85]:
for name, pipe, param_dist in model_spaces:

    print(f"\n================ {name} =================")

    search = OptunaSearchCV(
        estimator=pipe,
        param_distributions=param_dist,
        n_trials=n_trials_dict[name],
        scoring='roc_auc',
        cv=cv,                     
        n_jobs=1 if name == 'CatBoost' else -1,
        random_state=RANDOM_STATE,
        refit=True,
        error_score='raise'     
    )

    search.fit(
        X_train[features_to_model],
        y_train
    )

    results[name] = search

    print("Best CV AUC:", search.best_score_)
    print("Best params:")
    for k, v in search.best_params_.items():
        print(f"  {k}: {v}")



================ LogReg =================


C:\Temp\ipykernel_19292\1111529917.py:5: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  search = OptunaSearchCV(
[I 2026-02-09 23:12:34,759] A new study created in memory with name: no-name-10c613db-d5c8-49e9-b3c4-c6bd8afba211
[I 2026-02-09 23:13:30,888] Trial 3 finished with value: 0.7286189922127538 and parameters: {'model__C': 0.0004944678897998481}. Best is trial 3 with value: 0.7286189922127538.
[I 2026-02-09 23:13:31,795] Trial 6 finished with value: 0.7293714906590608 and parameters: {'model__C': 0.0011394968516426308}. Best is trial 6 with value: 0.7293714906590608.
[I 2026-02-09 23:13:35,764] Trial 7 finished with value: 0.729713053879162 and parameters: {'model__C': 0.0032938711790795653}. Best is trial 7 with value: 0.729713053879162.
[I 2026-02-09 23:14:39,006] Trial 1 finished with value: 0.7298354196933344 and parameters: {'model__C': 0.017634754723335578}. Best is trial 1 with value: 0.7298354196933

Best CV AUC: 0.7298420460963841
Best params:
  model__C: 0.9134740600120848

================ CatBoost =================


C:\Temp\ipykernel_19292\1111529917.py:5: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  search = OptunaSearchCV(
[I 2026-02-09 23:15:58,763] A new study created in memory with name: no-name-9c219751-de41-44ed-8925-ff504d4bffff
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
[I 2026-02-09 23:16:57,786] Trial 0 finished with value: 0.7592973642079665 and parameters: {'model__depth': 7, 'model__learning_rate': 0.13603856056466954, 'model__l2_leaf_reg': 1.3101353258498298, 'model__bagging_temperature': 0.31944725103976734, 'model__random_strength': 0.9703743859519267}. Best is trial 0 with value: 0.7592973642079665.
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Defau

Best CV AUC: 0.759868238919517
Best params:
  model__depth: 7
  model__learning_rate: 0.16190744442942542
  model__l2_leaf_reg: 41.961459292655185
  model__bagging_temperature: 0.10816978785158486
  model__random_strength: 0.6324678676427624

================ LightGBM =================


C:\Temp\ipykernel_19292\1111529917.py:5: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  search = OptunaSearchCV(
[I 2026-02-09 23:46:45,908] A new study created in memory with name: no-name-1ae51cdf-8f1c-4956-948b-e593719eb3d5
C:\Users\Андрей\AppData\Roaming\Python\Python39\site-packages\optuna_integration\sklearn\sklearn.py:398: RuntimeWarning: Mean of empty slice
  trial.set_user_attr("mean_{}".format(name), np.nanmean(array))
C:\Users\Андрей\AppData\Roaming\Python\Python39\site-packages\optuna_integration\sklearn\sklearn.py:398: RuntimeWarning: Mean of empty slice
  trial.set_user_attr("mean_{}".format(name), np.nanmean(array))
c:\ProgramData\anaconda3\envs\project_shift\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Андрей\AppData\Roaming\Python\Python39\site-packages\optuna_integration\

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U5'), dtype('<U5')) -> None

Полные данные

In [86]:
X_full = pd.concat([X_train, X_val], axis=0)
y_full = pd.concat([y_train, y_val], axis=0)

print(X_full.shape, y_full.shape)


(1210779, 130) (1210779,)


Лучшая модель (Все гиперпараметры я проверить не успел, так что выбрал наилучшую модель из найденных)

Для нее использовал увеличенное число итераций

Модель имела значение метрики `roc_auc` = 0.759868238919517 на кросс-валидации

In [88]:
best_model_params = {'depth': 7, 
                     'learning_rate': 0.16190744442942542, 
                     'l2_leaf_reg': 41.961459292655185, 
                     'bagging_temperature': 0.10816978785158486, 
                     'random_strength': 0.6324678676427624}

final_model = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='AUC',
    iterations=900,
    auto_class_weights='Balanced',
    random_seed=RANDOM_STATE,
    verbose=False,
    task_type='GPU',
    devices='0',
    cat_features=cat_feature_indices,
    **best_model_params
)


final_model.fit(
    X_full[features_to_model],
    y_full
)


Default metric period is 5 because AUC is/are not implemented for GPU


Предсказание на тестовой выборке

In [96]:
X_test = pd.read_csv('./shift_ml_2026_test.csv')
test_df = ptest_df.join(X_test['id'])

NameError: name 'ptest_df' is not defined

In [ ]:
test_pred = final_model.predict_proba(
    test_df[features_to_model]
)[:, 1]


KeyError: "['prof_te', 'first_loan_year', 'prof_group_te', 'prof_freq_enc'] not in index"

Формирование файла `submissions.csv`

In [98]:
submission = pd.DataFrame({
    'id': test_df.index,
    'target': test_pred
})

submission.to_csv('submission.csv', index=False)
print('submission.csv успешно сохранён')


submission.csv успешно сохранён
